In [132]:
import pandas as pd
import numpy as np
import graphviz
import xgboost as xgb

from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [133]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [134]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [135]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [136]:
train.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [137]:
train.head(n = 40)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [138]:
drop_columns = ['Name', 'PassengerId', 'Ticket']

pid_test = test['PassengerId']

datasets = [train, test]

In [139]:
for dataset in datasets:
    dataset.drop(drop_columns, axis = 1, inplace = True)
    
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
    
    dataset['Age'] = dataset['Age'].fillna(train['Age'].median())
    
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    dataset['Sex'] = dataset['Sex'].map( {'female' : 0, 'male' : 1} ).astype(int)
    
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    
    dataset['Has_Cabin'] = dataset["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    
    dataset.drop('Cabin', axis = 1, inplace = True)
    
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    
    dataset.drop(['SibSp', 'Parch'], axis = 1, inplace = True)
    
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;
    

In [140]:
train.isnull().sum()
train.sample(n = 10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Has_Cabin,FamilySize,IsAlone
337,1,1,0,2.0,3.0,1,1,1,1
233,1,3,0,0.0,3.0,0,0,7,0
299,1,1,0,3.0,3.0,1,1,2,0
522,0,3,1,1.0,0.0,1,0,1,1
488,0,3,1,1.0,1.0,0,0,1,1
157,0,3,1,1.0,1.0,0,0,1,1
369,1,1,0,1.0,3.0,1,1,1,1
645,1,1,1,2.0,3.0,1,1,2,0
389,1,2,0,1.0,1.0,1,0,1,1
189,0,3,1,2.0,0.0,0,0,1,1


In [141]:
dec_tree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 100, max_leaf_nodes = None)
gbm = xgb.XGBClassifier(n_estimators = 2000, max_depth = 4, min_child_weight = 2, gamma = 0.9, subsample = 0.8, 
                        colsample_bytree = 0.8, objective = 'binary:logistic', nthread = -1, scale_pos_weight = 1)

In [142]:
col_names = list(datasets[0])
X = train[[col for col in col_names if col != 'Survived']]
Y = train['Survived']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 0)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(668, 8) (223, 8) (668,) (223,)


In [143]:
for clf_name, clf in zip(['Decision Tree', 'XGB'], [dec_tree, gbm]):
    clf.fit(X_train, Y_train)
    
    Y_train_predicted = clf.predict(X_train)
    Y_test_predicted = clf.predict(X_test)
    #print('Y_train', Y_train, 'Y_train_predicted', Y_train_predicted)

    print(clf_name + '\n-------------')
    print('Training Score:{}%'.format((Y_train == Y_train_predicted).mean() * 100))
    print('Testing Score:{}%\n'.format((Y_test == Y_test_predicted).mean() * 100))

Decision Tree
-------------
Training Score:88.77245508982035%
Testing Score:80.26905829596413%

XGB
-------------
Training Score:85.92814371257485%
Testing Score:82.0627802690583%



C:\Users\HeadQuarters\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HeadQuarters\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [144]:
print("Feature importances :\n{}".format(clf.feature_importances_))

Feature importances :
[0.12667947 0.09788868 0.16276391 0.19769673 0.12629558 0.0537428
 0.20882918 0.02610365]


In [154]:
submission = np.asarray([pid_test, clf.predict(test)])

submission = submission.T

pd.DataFrame(submission).to_csv('Submission.csv')

C:\Users\HeadQuarters\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
export_graphviz(clf, out_file = 'tree.dot', class_names = ["Didn't survive", "Survived"], feature_names = X_train.columns.values, impurity = False, filled = True)

In [ ]:
with open('tree.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)